In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# api url
base_url = 'http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey=9oUviJPMmodfrNLBPYD0czH%2BOoPnArWcnHsw6T5g2TwnKEs1dwXCp%2FI%2BNfeEGngIAK74J550ZjxJ0CxJy4cK7Q%3D%3D&resultType=json'

# rsi 함수
def calc_stochastic(url, start_date):
    response = requests.get(url)
    
    if response.status_code == 200:
        contents = response.text  # JSON 데이터 가져오기
    else:
        print(f"API 호출 실패: {response.status_code}")
    
    json_ob = json.loads(contents)

    data = json_ob['response']['body']['items']['item']
    selected_data = []

    for item in data:
        selected_item = {
            'basDt': item['basDt'],
            'itmsNm': item['itmsNm'],
            'mrktCtg': item['mrktCtg'],
            'clpr': item['clpr'],
            'vs': item['vs'],
            'fltRt': item['fltRt'],
            'mkp': item['mkp'],
            'hipr': item['hipr'],
            'lopr': item['lopr']
        }
        selected_data.append(selected_item)
        
    dataframe = pd.DataFrame(selected_data)

    # 데이터 프레임 열 이름 변경
    dataframe = dataframe.rename(columns={
        'basDt': '날짜',
        'itmsNm': '종목명',
        'mrktCtg': '시장구분',
        'clpr': '종가',
        'vs': '거래량',
        'fltRt': '등락률',
        'mkp': '시가총액',
        'hipr': '고가',
        'lopr': '저가'
    })
    
    # 날짜를 날짜 형식으로 변환
    dataframe['날짜'] = pd.to_datetime(dataframe['날짜'], format='%Y%m%d')

    # 종가를 수치로 변환
    dataframe['종가'] = pd.to_numeric(dataframe['종가'])

    # 날짜를 인덱스로 설정
    dataframe.set_index('날짜', inplace=True)
    
    # 스토캐스틱 계산 (예: 14일 기간, 3일 D%)
    period = 14
    k_period = 3

    # 최고가(High), 최저가(Low), 종가(Close)를 사용하여 스토캐스틱 K, D 계산
    dataframe['Lowest_Low'] = dataframe['저가'].rolling(window=period).min()
    dataframe['Highest_High'] = dataframe['고가'].rolling(window=period).max()
    dataframe['Stochastic_K'] = (dataframe['종가'] - dataframe['Lowest_Low']) / (dataframe['Highest_High'] - dataframe['Lowest_Low']) * 100
    dataframe['Stochastic_D'] = dataframe['Stochastic_K'].rolling(window=k_period).mean()

    # 시각화
    plt.figure(figsize=(10, 5))
    # x축에 넣을 날짜 생성
    x_ticks = pd.date_range(start=start_date, periods=7, freq='2D')
    plt.plot(x_ticks, dataframe['Stochastic_K'], label='Stochastic K', marker='o', linestyle='-', color='b')
    plt.plot(x_ticks, dataframe['Stochastic_D'], label='Stochastic D', marker='x', linestyle='-', color='r')
    plt.axhline(y=80, color='r', linestyle='--', label='Overbought (80)')
    plt.axhline(y=20, color='g', linestyle='--', label='Oversold (20)')
    plt.title('Stochastic Oscillator')
    plt.xticks(rotation=45)  # x축 레이블에 45도 각도 적용
    plt.xlabel('Date')
    plt.ylabel('Stochastic Value')
    plt.legend()
    return plt.show()
